#### USCRN Data 

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import yaml 

with open ("sources.yaml", "r") as yaml_file:
  sources = yaml.load(yaml_file)

/tmp/ipykernel_56581/798438717.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  sources = yaml.load(yaml_file)


In [8]:
header_url = sources['USCRN']['headers']
header_response = requests.get(header_url)
header_soup = BeautifulSoup(header_response.content, "html.parser")

columns = str(header_soup).split("\n")[1].strip(" ").split(" ")
columns = list(map(lambda x: str.lower(x), columns)) # columns = [str.lower(c) for c in columns] -- faster?
columns.insert(0,'station_location')

In [9]:
base_url = sources["USCRN"]["index"]
base_response = requests.get(base_url)
base_soup = BeautifulSoup(base_response.content, "html.parser")

In [10]:
import re

links = base_soup.find_all("a") # 'links' in this notebook will refer to <a> elements, not urls
years = [str(x).zfill(1) for x in range(2000,2024)]
year_links = [link for link in links if link['href'].rstrip('/') in years]

file_urls = []
for year_link in year_links: 
  year_url = base_url + year_link.get("href")
  response = requests.get(year_url) 
  soup = BeautifulSoup(response.content, 'html.parser')
  file_links = soup.find_all('a', href=re.compile(r'AK.*\.txt'))
  if file_links:
    new_file_urls = [year_url + link.getText() for link in file_links]
    file_urls.extend(new_file_urls)

In [146]:
rows = []
for url in file_urls:
  # Get location from url -- will add to BS results in next step
  regex = r"([St.]*[A-Z][a-z]+_*[A-Za-z]*).*.txt" 
  file_name = re.search(regex, url).group(0)
  station_location = re.sub("(_formerly_Barrow.*|_[0-9].*)", "", file_name)
  # Get results 
  response = requests.get(url)
  soup = BeautifulSoup(response.content,'html.parser')
  soup_lines = [station_location + " " + line for line in str(soup).split("\n")]
  new_rows = [re.split('\s+', row) for row in soup_lines]
  rows.extend(new_rows)

In [148]:
df = pd.DataFrame(rows)

In [151]:
df[~df[39].isnull()]

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
54113,St.,Paul,25711,20050807,2300,20050807,1400,1.201,-170.21,57.16,...,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None
54114,St.,Paul,25711,20050808,0000,20050807,1500,1.201,-170.21,57.16,...,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None
54115,St.,Paul,25711,20050808,0100,20050807,1600,1.201,-170.21,57.16,...,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None
54116,St.,Paul,25711,20050808,0200,20050807,1700,1.201,-170.21,57.16,...,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None
54117,St.,Paul,25711,20050808,0300,20050807,1800,1.201,-170.21,57.16,...,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159669,Toolik,Lake,96409,20230213,2100,20230213,1200,2.514,-149.40,68.65,...,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None
2159670,Toolik,Lake,96409,20230213,2200,20230213,1300,2.514,-149.40,68.65,...,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None
2159671,Toolik,Lake,96409,20230213,2300,20230213,1400,2.514,-149.40,68.65,...,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None
2159672,Toolik,Lake,96409,20230214,0000,20230213,1500,2.514,-149.40,68.65,...,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,None
